<a href="https://colab.research.google.com/github/nhwhite212/DealingwithDataSpring2021/blob/master/6-Pandas/G-Pandas_Misc_MapFunction_Plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We start this notebook by typing a "magic" command that allows iPython notebooks to display plots directly in the browser.

In [ ]:
# Render our plots inline
%matplotlib inline

In order to read and process files, we are going to use a very powerful, and widely used Python library, called pandas. So, our next step is to import the pandas library in Python, and also import the library matplotlib for generating plots:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Pandas should already be installed on your machine, but if you get an error in the import statement above, indicating that pandas is not available, please uncomment the following command and execute:

In [ ]:
# !sudo python3 -m pip install -U pandas

And we type some code to simply change the visual style of the plots. (The code below is optional and not necessary, and for now you do not need to understand what exactly is happening.)

In [ ]:
# Make the graphs a bit bigger
plt.rcParams['figure.figsize'] = (15, 5)

### Basic Example: Creating a DataFrame from Scratch

This is an example of creating a dataframe by passing a list of dictionaries.

In [ ]:
df = pd.DataFrame([ 
    {"First Name": "Panos", "Last Name":" Ipeirotis"},
    {"First Name": "John", "Last Name":" Doe"},
    {"Last Name":"Trump"}
])


In [ ]:
df

### Reading Data Files

Let's take a look at the restaurant inspections file (at `data/restaurants.csv`), which we used in our earlier classes.

If you do not have it, then uncomment and execute the following:

In [ ]:
!mkdir data
# NOTE: THE API NO LONGER INCLUDES DESCRIPTIONS, YOU NEED THE FULL DATASET
#!curl 'https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD' -o data/restaurant.csv
!curl http://people.stern.nyu.edu/nwhite/DealingwithDataSpring2021/data/restaurant.csv.gz -o data/restaurant.csv.gz


In [ ]:
#!rm data/restaurant.csv.gz
#!gzip data/restaurant.csv

In [ ]:
!gzip -cd data/restaurant.csv.gz | head -3

### Reading a CSV file

We want to be able to read and process this file within Python. The pandas library has a very convenient method `read_csv` which reads the file, and returns back a variable that contains its contents.

In [ ]:
import pandas as pd
restaurants = pd.read_csv("data/restaurant.csv.gz", encoding="utf-8", dtype="unicode")

When you read a CSV, you get back a kind of object called a DataFrame, which is made up of rows and columns. You get columns out of a DataFrame the same way you get elements out of a dictionary. Let's take a look at how the object looks like:

In [ ]:
restaurants.head(5)

The read_csv method has many options, and you can read further in the [online documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.parsers.read_csv.html).

### Descriptive statistics

We can use the method "describe()" to get a quick overview of the data in the dataframe.

In [ ]:
restaurants.describe()

We can also check the data types for each column

In [ ]:
restaurants.dtypes

The `object` type is a string. For many of these, we would like to change the data types for a few columns, using the `pd.to_numeric` and `pd.to_datetime` functions:

In [ ]:
restaurants["SCORE"] = pd.to_numeric(restaurants["SCORE"])
restaurants.dtypes

In [ ]:
restaurants[["SCORE"]].describe()

Now let's convert the dates columns into the appropriate data types:

In [ ]:
restaurants["GRADE DATE"] = pd.to_datetime(restaurants["GRADE DATE"], format="%m/%d/%Y")
restaurants["RECORD DATE"] = pd.to_datetime(restaurants["RECORD DATE"], format="%m/%d/%Y")
restaurants["INSPECTION DATE"] = pd.to_datetime(restaurants["INSPECTION DATE"], format="%m/%d/%Y")

In [ ]:
restaurants.dtypes

In [ ]:
restaurants[["INSPECTION DATE", "GRADE DATE", "RECORD DATE"]].describe()

In [ ]:
restaurants["BORO"] =  pd.Categorical(restaurants["BORO"], ordered=False)
restaurants["GRADE"] =  pd.Categorical(restaurants["GRADE"], categories = ['A', 'B', 'C'], ordered=True)
restaurants["VIOLATION CODE"] =  pd.Categorical(restaurants["VIOLATION CODE"], ordered=False)
restaurants["CRITICAL FLAG"] =  pd.Categorical(restaurants["CRITICAL FLAG"], ordered=False)
restaurants["ACTION"] =  pd.Categorical(restaurants["ACTION"], ordered=False)
restaurants["CUISINE DESCRIPTION"] =  pd.Categorical(restaurants["CUISINE DESCRIPTION"], ordered=False)
restaurants.dtypes

In [ ]:
restaurants.describe(include='all')

### Selecting a subset of the columns

In a dataframe, we can specify the column(s) that we want to keep, and get back another dataframe with just the subset of the columns that we want to keep.

In [ ]:
restaurants["VIOLATION CODE"].head(5)

In [ ]:
column_list = ["GRADE DATE","VIOLATION CODE", "DBA"]
restaurants[ column_list ].head(5)

We can also get quick statistics about the common values that appear in each column:

In [ ]:
restaurants["DBA"].value_counts()

In [ ]:
restaurants["CUISINE DESCRIPTION"].value_counts()[:20]

In [ ]:
violation_counts = restaurants["VIOLATION CODE"].value_counts();
violation_counts[0:10]

In [ ]:
violation_counts = restaurants["VIOLATION DESCRIPTION"].value_counts();
violation_counts[0:10]

And we can use the "plot" command to plot the resulting histogram. (More details at http://pandas.pydata.org/pandas-docs/stable/visualization.html) We also revisit plotting below.

In [ ]:
violation_counts[:5].plot(kind='bar')

In [ ]:
popular = restaurants["CUISINE DESCRIPTION"].value_counts()
popular[:10].plot(kind='bar')

### Using the map command

The map command in Python has the following syntax:

`map(function, [list of values for first argument], [list of values for second argument]...)`

It takes as input a function, which has a set of parameters. Then, it iterates over the lists that follow; the lists contain the arguments that are passed to the function. Map returns a list of values that are the result of applying the function to all the elements of the list(s). 

For example, in the following code, the `add` function is going to be applied to the two lists (`[1, 2, 3, 4]` and `[9, 10, 10, 11]`) that follow. The result of the map will be a list containing the values `[add(1,2), add(2,10), add(3,10), add(4,11)]`




In [ ]:
def capitalize(x):
    return x[0].upper() + x[1:].lower()

capitalize("panos")

In [ ]:
example = map(capitalize, ["panos", "christopher", "caterina", "lucy"])
list(example)

In [ ]:
def add(x,y):
    return x+y
    
example = map(add, [1, 2, 3, 4], [9, 10, 10, 11])
list(example)

#### Using the map for dataframes

Using the map command, we can:
* Create new columns for the dataframe
* Modify existing columns
* Generate new columns that are the result of operations on the columns of the dataframe

For example, suppose that we want to format the phone column. We can write a function that takes as input a phone and formats it as we want. Then we apply the function using the map command as follows:

In [ ]:
restaurants['PHONE']

In [ ]:
import re
def formatPhone(phoneString):
    if phoneString == None:
        return ""
    regex = re.compile(r'([2-9]\d{2})\W*(\d{3})\W*(\d{4})')
    match = regex.search(str(phoneString))
    if match:
        area_code = match.group(1)
        first_three = match.group(2)
        last_four = match.group(3)
        formatted = "(" + area_code + ") " + first_three + "-" + last_four
        return formatted
    else:
        return ""

In [ ]:
formatPhone("2129980803")

In [ ]:
restaurants['FormattedPhone'] = restaurants['PHONE'].apply(formatPhone)

In [ ]:
list(map(formatPhone, restaurants['PHONE']))

In [ ]:
restaurants[['PHONE', 'FormattedPhone']]

In [ ]:
def printAddress(boro, building, street, zipcode, phone):
    if boro == None:
        boro = ""
    if building == None:
        building = ""
    if street == None:
        street = ""
    if zipcode == None:
        zipcode = ""
    if phone == None:
        phone = ""
    
    result = str(building) + " " + str(street) + ", " + \
    str(boro) + ", " + str(zipcode) +  ", Phone: " + str(formatPhone(phone))
    return result

printAddress("Manhattan", "44", "West 4th", "10012", 2129980803
             
             
            )


In [ ]:

restaurants["FormattedAddress"] = list(map(printAddress, 
                                     restaurants["BORO"],
                                     restaurants["BUILDING"],
                                     restaurants["STREET"],
                                     restaurants["ZIPCODE"],
                                     restaurants["PHONE"]))

In [ ]:
restaurants.head(10)

In [ ]:
restaurants[["FormattedAddress", "STREET", "BUILDING", "BORO"]]


### Selecting rows

To select rows, we can use the following approach, where we generate a list of boolean values, one for each row of the dataframe, and then we use the list to select which of the rows of the dataframe we want to keep"

In [ ]:
# Selecting columns from a data frame: create a list of column
# names and pass it within brackets [ ]
restaurants[  ["GRADE DATE","VIOLATION CODE", "DBA"] ].head(5)

In [ ]:
restaurants

In [ ]:
# Create the condition "has mice" for code 04L
mice = (restaurants["VIOLATION CODE"] == "04L")
mice

In [ ]:

# Apply the condition to the dataframe "restaurants" and store the result 
# in a dataframe called  has_mice
has_mice = restaurants[mice]
has_mice

In [ ]:
# List the most frequent DBA values in the dataframe
has_mice["DBA"].value_counts()[:20]

In [ ]:
has_mice["CAMIS"].value_counts()[:5]

In [ ]:
condition = (restaurants["CAMIS"] == '41485393')
restaurants[condition]


And we can use more complex conditions:

In [ ]:
mice_manhattan = (restaurants["VIOLATION CODE"] == "04L") \
                    & (restaurants["BORO"] == "MANHATTAN") \
                    & (restaurants["ZIPCODE"] == "10012")

In [ ]:
has_mice_10012 = restaurants[mice_manhattan]
has_mice_10012

In [ ]:
has_mice_10012["DBA"].value_counts()[:20]

In [ ]:
has_mice_10012["DBA"].value_counts()[:100].plot(kind='bar')

### Plotting

In [ ]:
# We need to convert the "SCORE" column to a numeric data type (just in case the related part was not executed above)
restaurants["SCORE"] = pd.to_numeric(restaurants["SCORE"])

In [ ]:
# We can plot a histogram of the scores using the plot(kind='hist') command
restaurants["SCORE"].plot(kind='hist', # histogram
                          bins=50, # use 50 bars
                          alpha=0.5, # add a bit of transparency
                          range=(0,50), # x-axis from 0 to 50
                          color='Blue',# use Blue color
                          density=False)  # show raw count, density = False

In [ ]:
# This plots the density of a numeric attribute
# kde = kernel density estimation
restaurants["SCORE"].plot(kind='kde', color='Black', xlim=(0,50))

In [ ]:
# This plots the density of a numeric attribute
# kde = kernel density estimation of the probability density function
# We can plot a histogram of the scores using the plot(kind='hist') command
Norm=(1==1)
restaurants["SCORE"].plot(kind='hist', # histogram
                          bins=50, # use 50 bars
                          alpha=0.5, # add a bit of transparency
                          range=(0,50), # x-axis from 0 to 50
                          color='Blue', # use blue color
                          density=True) # show the density, area adds up to 1.0
restaurants["SCORE"].plot(kind='kde', color='Black', xlim=(0,50))

### Look at only the "A" Grade restaurants

In [ ]:
restaurants["SCORE"][ restaurants["GRADE"] == 'A' ].plot(kind = 'hist', 
                                                         bins=50, 
                                                         range=(0,50), 
                                                         color='Green', 
                                                         density=True)

#### and only the B's

In [ ]:
restaurants["SCORE"][ restaurants["GRADE"] == 'B' ].plot(kind = 'hist', 
                                                         bins=50, 
                                                         range=(0,50), 
                                                         color='Orange', 
                                                         density=True)

### and the C's  (Do you see a problem here?)

In [ ]:
restaurants["SCORE"][ restaurants["GRADE"] == 'C' ].plot(kind = 'hist', 
                                                         bins=50, 
                                                         range=(0,50), 
                                                         color='Red', 
                                                         density=True)

# All Together

In [ ]:
# We can also select specific rows to plot (e.g., in this case based on grade)
# Jupyter is smart enough to combine these three plots in one 
# (sometimes it is not smart, below we show how to make our desire to combine plots explicit)
restaurants["SCORE"][ restaurants["GRADE"] == 'A' ].plot(kind = 'hist', bins=50, range=(0,50), color='Green', density=True)
restaurants["SCORE"][ restaurants["GRADE"] == 'B' ].plot(kind = 'hist', bins=50, range=(0,50), color='Orange', density=True)
restaurants["SCORE"][ restaurants["GRADE"] == 'C' ].plot(kind = 'hist', bins=50, range=(0,50), color='Red', density=True)

#### and make the Grades partially transparent so we can show the full density function

In [ ]:
ax1 = restaurants["SCORE"][ restaurants["GRADE"] == 'A' ].plot(kind = 'hist',bins=50, alpha=0.5, range=(0,50), color='Green', density=True)
ax2 = restaurants["SCORE"][ restaurants["GRADE"] == 'B' ].plot(kind = 'hist',bins=50, alpha=0.5, range=(0,50), color='Orange', density=True, ax=ax1)
ax3 = restaurants["SCORE"][ restaurants["GRADE"] == 'C' ].plot(kind = 'hist',bins=50, alpha=0.5, range=(0,50), color='Red', density=True, ax=ax2)
restaurants["SCORE"].plot(kind='kde', color='Black', xlim=(0,50), ax=ax3)

# USE SQL ON Datframes
### Comparison with SQL

For a comparison with SQL, see http://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html

Pandas supports its own set of operators for doing SQL-like operations (in reality it runs a in-memory SQL database in the backend). It is also possible to use straightforward SQL queries to query Pandas dataframes, by using the `pandasql` package:

In [ ]:
!sudo -H pip install -U pandasql

In [ ]:
from pandasql import sqldf

In [ ]:
# PandaSQL does not like column names with spaces. So we will rename (some of) them.
restaurants.rename(columns={"VIOLATION CODE": "VIOLATION"}, inplace = True)

In [ ]:
rest = restaurants[["DBA", "BORO", "VIOLATION", "SCORE"]]
# only take first 5000 as an example, SEEMS TO BLOW UP AT AROUND 7500
nrest=rest.head(50000)

In [ ]:
q  = """
SELECT BORO, VIOLATION, COUNT(*) AS CNT 
FROM
  nrest
GROUP BY BORO, VIOLATION
ORDER BY CNT DESC
LIMIT 20;
"""

df = sqldf(q, globals())

In [ ]:
df

In [ ]:
q  = """
SELECT SCORE, COUNT(*) AS CNT 
FROM
  nrest
GROUP BY SCORE
ORDER BY SCORE
"""

df = sqldf(q, globals())

In [ ]:
df

In [ ]:
df.plot(kind='scatter', s=50, x="SCORE", y="CNT", xlim=(0,30))